# Profit分析

## 目的

予測での利益と実際の利益の差分をみてみる。

In [ ]:
import os
import datetime
import numpy as np
import pandas as pd

In [ ]:
current_dir = %pwd
timestamp = os.path.basename(current_dir)

In [ ]:
import sys
sys.path.append('../../')
%cd '../../'

In [ ]:
from src.utils.trade_analysis import TradeAnalysis

%reload_ext autoreload
%autoreload 2

In [ ]:
ta = TradeAnalysis(timestamp)

In [ ]:
cc_trades = ta.get_coincheck_trades_df()
lq_trades = ta.get_liquid_trades_df()

In [ ]:
len(cc_trades), len(lq_trades)

In [ ]:
cc_ticks = ta.get_coincheck_ticks_df()
lq_ticks = ta.get_liquid_ticks_df()

In [ ]:
cc_ticks.head(5)

In [ ]:
trades = []

for cc_row, lq_row in zip(cc_trades.iterrows(), lq_trades.iterrows()):
    cc_item = cc_row[1]
    lq_item = lq_row[1]
    
    cc_side = cc_item["side"]
    cc_timestamp = cc_item["datetime"]
    cc_amount = cc_item["amount"]
    cc_actual_price = cc_item["price"]
    cc_actual_rate = cc_item["rate"]

    lq_side = lq_item["side"]
    lq_timestamp = lq_item["datetime"]
    lq_amount = lq_item["amount"]
    lq_actual_price = lq_item["price"]
    lq_actual_rate = lq_item["rate"]
    
    try:
        cc_tick = cc_ticks.loc[cc_timestamp]
    except Exception:
        timestamp = cc_timestamp + datetime.timedelta(seconds=1)
        try:
            cc_tick = cc_ticks.loc[timestamp]
        except Exception:
            timestamp2 = timestamp + datetime.timedelta(seconds=1)
            cc_tick = cc_ticks.loc[timestamp2]

    try:
        lq_tick = lq_ticks.loc[lq_timestamp]
    except Exception:
        timestamp = lq_timestamp + datetime.timedelta(seconds=1)
        try:
            lq_tick = lq_ticks.loc[timestamp]
        except Exception:
            timestamp2 = timestamp + datetime.timedelta(seconds=1)
            lq_tick = lq_ticks.loc[timestamp2]

    if cc_side == "sell":
        cc_tick_rate = cc_tick.bid
    else:
        cc_tick_rate = cc_tick.ask
    if lq_side == "sell":
        lq_tick_rate = lq_tick.bid
    else:
        lq_tick_rate = lq_tick.ask
        
    cc_expected_price = cc_tick_rate * cc_amount
    lq_expected_price = lq_tick_rate * lq_amount

    
    if cc_side == "buy":
        cc_expected_price = cc_expected_price * -1
        cc_actual_price = cc_actual_price * -1
    if lq_side == "buy":
        lq_expected_price = lq_expected_price * -1
        lq_actual_price = lq_actual_price * -1
        
    expected_profit = sum([cc_expected_price, lq_expected_price])
    actual_profit = sum([cc_actual_price, lq_actual_price])

    trade = {
        "timestamp": cc_timestamp, 
        "cc_side": cc_side,
        "cc_expected_rate": cc_tick_rate,
        "cc_actual_rate": cc_actual_rate,
        "cc_expected_price": cc_expected_price,
        "cc_actual_price": cc_actual_price,
        "lq_side": lq_side,
        "lq_expected_rate": lq_tick_rate,
        "lq_actual_rate": lq_actual_rate,
        "lq_expected_price": lq_expected_price,
        "lq_actual_price": lq_actual_price,
        "expected_profit": expected_profit,
        "actual_profit": actual_profit
    }
    
    trades.append(trade)

In [ ]:
trade_df = pd.DataFrame(trades)
trade_df = trade_df.set_index("timestamp")

In [ ]:
profit_df = trade_df[["expected_profit", "actual_profit"]]

In [ ]:
int(sum(profit_df["expected_profit"])), int(sum(profit_df["actual_profit"]))

In [ ]:
pd.set_option('display.max_rows', None)
profit_df

## coincheck buyにおける売却コインの差分分析

In [ ]:
cc_buy_trades = cc_trades[cc_trades["side"] == "buy"]

In [ ]:
cc_buy_amount = cc_buy_trades["amount"]
min(cc_buy_amount), max(cc_buy_amount), np.mean(cc_buy_amount)

In [ ]:
from decimal import Decimal
num = 0.006 - np.mean(cc_buy_amount)
"{0:.10f}".format(Decimal(num))

この価格差によって生じた資金移動は

In [ ]:
num * len(cc_buy_trades) * np.mean(cc_ticks["ask"])

## rateの誤差比較

In [ ]:
cc_rate_df = trade_df[["cc_side", "cc_expected_rate", "cc_actual_rate"]].copy()
cc_rate_df["cc_rate_diff"] = cc_rate_df["cc_expected_rate"] - cc_rate_df["cc_actual_rate"]

In [ ]:
cc_rate_df

In [ ]:
cc_diff = cc_rate_df["cc_rate_diff"]
max(cc_diff), min(cc_diff), np.mean(abs(cc_diff)), np.std(cc_diff)

In [ ]:
lq_rate_df = trade_df[["lq_side", "lq_expected_rate", "lq_actual_rate"]].copy()
lq_rate_df["lq_rate_diff"] = lq_rate_df["lq_expected_rate"] - lq_rate_df["lq_actual_rate"]

In [ ]:
lq_diff = lq_rate_df["lq_rate_diff"]
max(lq_diff), min(lq_diff), np.mean(abs(lq_diff)), np.std(lq_diff)